In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import math
import GraphModule as gm

In [ ]:
def plot_chart (df, title, x_label, y_label, chart_type, stack_yn) :

    fig = plt.figure(figsize=(10, 5), dpi=200)
    ax = plt.gca()
    # 차트 제목
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    df.plot(kind=chart_type if chart_type else 'bar', ax=ax, 
            stacked = stack_yn if stack_yn is False else True)
    return fig

In [ ]:
def hist_chart(df, title : str, x_label : str, y_label : str, options : dict) :

    fig = plt.figure(figsize=(10, 5), dpi=200)
    ax = plt.gca()
    # 차트 제목
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.hist(df['any'], bins = options['bins'] if options['bins'] else 10, 
                density=options['desity'] if options['desity'] else True, 
                color=options['color'] if options['color'] else 'blue', 
                histtype=options['hist_type'] if options['hist_type'] else 'bar')
    return fig

In [ ]:
data_loan_outstanding = pd.read_pickle(r'C:\Users\perkw\Projects\pqr-project-01\data\data_loan_outstanding.pkl')

In [ ]:
data_loan_outstanding.columns

In [ ]:
data_loan_outstanding_new = data_loan_outstanding[data_loan_outstanding['실행사유'] == '신규']

In [ ]:
data_loan_outstanding_new.shape

In [ ]:
data_loan_outstanding_new = data_loan_outstanding_new[data_loan_outstanding_new['이전실행번호'].isnull()]

In [ ]:
org_list = data_loan_outstanding_new['부서'].unique()
org_list

In [ ]:
filter_org = ['데이터금융1팀', '리스할부운영팀', '리스할부영업팀','데이터금융2팀']

In [ ]:
data_loan_outstanding_new = data_loan_outstanding_new[data_loan_outstanding_new['부서'].isin(filter_org)]
data_loan_outstanding_new.shape

In [ ]:
data_loan_outstanding_new = data_loan_outstanding_new[data_loan_outstanding_new['실행상태'].isin(['정상', '기한이익', '해지'])]
data_loan_outstanding_new.shape

In [ ]:
new_amt_retail_loan = pd.crosstab(data_loan_outstanding_new['vin_mob'], data_loan_outstanding_new['상품분류'], data_loan_outstanding_new['실행번호'], aggfunc='count')

In [ ]:
new_amt_retail_loan = new_amt_retail_loan.drop(['오토자금대출'], axis=1)
col_sort_list = ['오토리스', '할부/오토론', '임차보증금대출', '스탁론(국내)', '스마트스토어대출', '데이터_기타']
new_amt_retail_loan = new_amt_retail_loan[col_sort_list]
new_amt_retail_loan

In [ ]:
#pdf 생성
mypdf = PdfPages('pqr_chart_20210606.pdf')
# 파라미터 설정
plt.rcParams.update({'font.size': 10})
plt.rcParams.update({'font.family': 'Gulim'})

In [ ]:
# df, title : str, x_label : str, y_label : str, options : dict
# option : type, stack

fig = plot_chart(new_amt_retail_loan, '월간 리테일여신 건별 취급추이', '취급월', '취급건수(건)', 'bar', True)

In [ ]:
new_amt_retail_loan_amt = pd.crosstab(data_loan_outstanding_new['vin_mob'], data_loan_outstanding_new['상품분류'], data_loan_outstanding_new['여신금액']/100000000, aggfunc='sum')
new_amt_retail_loan_amt = new_amt_retail_loan_amt.drop(['오토자금대출', '데이터_기타'], axis=1)
col_sort_list = ['오토리스', '할부/오토론', '임차보증금대출', '스탁론(국내)', '스마트스토어대출']
new_amt_retail_loan_amt = new_amt_retail_loan_amt[col_sort_list]
fig = plot_chart(new_amt_retail_loan_amt, '월간 리테일여신 취급액(억원) 추이', '취급월', '취급액(억원)', 'bar', True)

In [ ]:
new_amt_retail_loan_mean = pd.crosstab(data_loan_outstanding_new['vin_mob'], data_loan_outstanding_new['상품분류'], data_loan_outstanding_new['여신금액']/1000000, aggfunc='mean')
new_amt_retail_loan_mean = new_amt_retail_loan_mean.drop(['오토자금대출', '데이터_기타'], axis=1)
col_sort_list = ['오토리스', '할부/오토론', '임차보증금대출', '스탁론(국내)', '스마트스토어대출']
new_amt_retail_loan_mean = new_amt_retail_loan_mean[col_sort_list]
fig = plot_chart(new_amt_retail_loan_mean, '월간 리테일여신 평균 Ticket size 추이', '취급월', '평균 취급 TS', 'line', False)
plt.ylim(0, 200)

In [ ]:
new_amt_retail_loan_mean_rate = pd.crosstab(data_loan_outstanding_new['vin_mob'], data_loan_outstanding_new['상품분류'], data_loan_outstanding_new['IRR'], aggfunc='mean')
new_amt_retail_loan_mean_rate = new_amt_retail_loan_mean_rate.drop(['오토자금대출', '데이터_기타'], axis=1)
col_sort_list = ['오토리스', '할부/오토론', '임차보증금대출', '스탁론(국내)', '스마트스토어대출']
new_amt_retail_loan_mean_rate = new_amt_retail_loan_mean_rate[col_sort_list]
fig = plot_chart(new_amt_retail_loan_mean_rate, '월간 리테일여신 평균대출금리(IRR) 추이', '취급월', '평균 IRR', 'line', False)
plt.ylim(2, 8)

In [ ]:
monthly_loan_outstanding = pd.read_pickle(r'C:\Users\perkw\Projects\pqr-project-01\data\monthly_loan_outstanding_f.pkl')
monthly_delinquent = pd.read_pickle(r'C:\Users\perkw\Projects\pqr-project-01\data\monthly_delinquent.pkl')

In [ ]:
monthly_loan_outstanding_merged = pd.merge(data_loan_outstanding, monthly_loan_outstanding, right_on = '실행번호', left_on = '실행번호', how='right')

In [ ]:
monthly_loan_outstanding.shape

In [ ]:
monthly_loan_outstanding_merged['여신잔액_y'] = monthly_loan_outstanding_merged['여신잔액_y'].str.replace(",", "")
monthly_loan_outstanding_merged['여신잔액_y'] = monthly_loan_outstanding_merged['여신잔액_y'].astype(float)

In [ ]:
monthly_loan_outstanding_merged['기준월'] = monthly_loan_outstanding_merged['기준일자'].apply(lambda x: x[0:6])

In [ ]:
monthly_loan_outstanding_merged.info()

In [ ]:
monthly_loan_outstanding_merged.to_excel('test.xlsx')

In [ ]:
monthly_loan_outstanding_merged_amt = monthly_loan_outstanding_merged[monthly_loan_outstanding_merged['부서'].isin(filter_org)]

In [ ]:
monthly_loan_outstanding_merged_amt_group = pd.crosstab(monthly_loan_outstanding_merged_amt['기준월'], monthly_loan_outstanding_merged_amt['상품분류'], monthly_loan_outstanding_merged_amt['여신잔액_y']/100000000, aggfunc='sum')

monthly_loan_outstanding_merged_amt_group = monthly_loan_outstanding_merged_amt_group.drop(['오토자금대출', '데이터_기타'], axis=1)
col_sort_list = ['오토리스', '할부/오토론', '임차보증금대출', '스탁론(국내)', '스마트스토어대출']
monthly_loan_outstanding_merged_amt_group = monthly_loan_outstanding_merged_amt_group[col_sort_list]
fig = plot_chart(monthly_loan_outstanding_merged_amt_group, '월간 리테일여신 여신잔액(억원) 추이', '월(month)', '여신잔액(억원)', 'bar', True)